In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD-2l.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 2.2213
Validation accuracy: 0.5662, validation loss: 2.1283
Epoch 2
Training loss: 2.0220
Validation accuracy: 0.7180, validation loss: 1.9077
Epoch 3
Training loss: 1.7774
Validation accuracy: 0.7513, validation loss: 1.6458
Epoch 4
Training loss: 1.5109
Validation accuracy: 0.7717, validation loss: 1.3867
Epoch 5
Training loss: 1.2696
Validation accuracy: 0.7918, validation loss: 1.1711
Epoch 6
Training loss: 1.0792
Validation accuracy: 0.8087, validation loss: 1.0084
Epoch 7
Training loss: 0.9380
Validation accuracy: 0.8247, validation loss: 0.8891
Epoch 8
Training loss: 0.8340
Validation accuracy: 0.8312, validation loss: 0.8010
Epoch 9
Training loss: 0.7562
Validation accuracy: 0.8401, validation loss: 0.7342
Epoch 10
Training loss: 0.6964
Validation accuracy: 0.8486, validation loss: 0.6820
Epoch 11
Training loss: 0.6492
Validation accuracy: 0.8529, validation loss: 0.6405
Epoch 12
Training loss: 0.6112
Validation accuracy: 0.8582, validation loss: 0.6066
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8950, device='cuda:0'), tensor(0.3845, device='cuda:0'))
Valid: (tensor(0.8931, device='cuda:0'), tensor(0.4017, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9041, device='cuda:0'), tensor(0.3308, device='cuda:0'))
Valid: (tensor(0.9019, device='cuda:0'), tensor(0.3513, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9041, device='cuda:0'), tensor(0.3308, device='cuda:0'))
Valid: (tensor(0.9019, device='cuda:0'), tensor(0.3513, device='cuda:0'))


## Epoch average

In [17]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [18]:
log_file = open("SGD-2l-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch_avg")

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 2.2226
Validation accuracy: 0.5534, validation loss: 2.1378
Epoch 2
Training loss: 2.0376
Validation accuracy: 0.6856, validation loss: 1.9303
Epoch 3
Training loss: 1.8037
Validation accuracy: 0.7319, validation loss: 1.6763
Epoch 4
Training loss: 1.5406
Validation accuracy: 0.7672, validation loss: 1.4161
Epoch 5
Training loss: 1.2951
Validation accuracy: 0.7941, validation loss: 1.1940
Epoch 6
Training loss: 1.0980
Validation accuracy: 0.8126, validation loss: 1.0249
Epoch 7
Training loss: 0.9510
Validation accuracy: 0.8274, validation loss: 0.9007
Epoch 8
Training loss: 0.8429
Validation accuracy: 0.8367, validation loss: 0.8090
Epoch 9
Training loss: 0.7622
Validation accuracy: 0.8429, validation loss: 0.7399
Epoch 10
Training loss: 0.7005
Validation accuracy: 0.8502, validation loss: 0.6862
Epoch 11
Training loss: 0.6521
Validation accuracy: 0.8553, validation loss: 0.6437
Epoch 12
Training loss: 0.6131
Validation accuracy: 0.8597, validation loss: 0.6090
E

In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8950, device='cuda:0'), tensor(0.3847, device='cuda:0'))
Valid: (tensor(0.8918, device='cuda:0'), tensor(0.4023, device='cuda:0'))


In [21]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [22]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [23]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8990, device='cuda:0'), tensor(0.3715, device='cuda:0'))
Valid: (tensor(0.8954, device='cuda:0'), tensor(0.3892, device='cuda:0'))


In [24]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [25]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [26]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8990, device='cuda:0'), tensor(0.3720, device='cuda:0'))
Valid: (tensor(0.8954, device='cuda:0'), tensor(0.3897, device='cuda:0'))
